In [1]:
import taichi as ti
import taichi.math as tm

[Taichi] version 1.7.2, llvm 15.0.4, commit 0131dce9, linux, python 3.9.19


[I 08/27/24 22:13:56.008 4004400] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


In [ ]:
!export CUDA_VISIBLE_DEVICES=0

ti.init(arch=ti.cuda)

[Taichi] Starting on arch=cuda


In [3]:
n = 128
dt = 4e-2 / n
# x is an n x n field consisting of 3D floating-point vectors
# representing the mass points' positions
x = ti.Vector.field(3, dtype=float, shape=(n, n))
# v is an n x n field consisting of 3D floating-point vectors
# representing the mass points' velocities
v = ti.Vector.field(3, dtype=float, shape=(n, n))

#? WHat's difference between ti.Vector.field and ti.field?

In [4]:
# The n x n grid is normalized
# The distance between two x- or z-axis adjacent points is 1.0 / n
quad_size = 1.0 / n

# The @ti.kernel decorator instructs Taichi to
# automatically parallelize all top-level for loops inside initialize_mass_points()
@ti.kernel
def initialize_mass_points():
    # A random offset to apply to each mass point
    random_offset = ti.Vector([ti.random() - 0.5, ti.random() - 0.5]) * 0.1

    # Field x stores the mass points' positions
    for i, j in x:
        # The piece of cloth is 0.6 (y-axis) above the original point
        #
        # By taking away 0.5 from each mass point's [x,z] coordinate value
        # you move the cloth right above the original point
        x[i, j] = [
            i * quad_size - 0.5 + random_offset[0], 0.6,
            j * quad_size - 0.5 + random_offset[1]
        ]
        # The initial velocity of each mass point is set to 0
        v[i, j] = [0, 0, 0]

In [5]:
ball_radius = 0.3
# Use a 1D field for storing the position of the ball center
# The only element in the field is a 3-dimentional floating-point vector
ball_center = ti.Vector.field(3, dtype=float, shape=(1, ))
# Place the ball center at the original point
ball_center[0] = [0, 0, 0]

In [9]:
# Gravity is a force applied in the negative direction of the y axis,
# and so is set to [0, -9.8, 0]
gravity = ti.Vector([0, -9.8, 0])

# The @ti.kernel decorator instructs Taichi to
# automatically parallelize all top-level for loops
# inside substep()
@ti.kernel
def substep():
    # The for loop iterates over all elements of the field v
    for i in ti.grouped(x):
        v[i] += gravity * dt

"""
`for i in ti.grouped(x)`
It means that this for loop automatically traverses all the elements of x as a 1D array regardless of its shape
"""

@ti.kernel
def test_grouped():
    for i in ti.grouped(x):
        print(f"the indices of x are {i}")

test_grouped()
#? Why can't use python print function in the kernel function?